# Synthetic FlyPCA Demo
This notebook mirrors the CLI pipeline on a synthetic dataset, illustrating lag-embedded PCA, feature extraction, and clustering.

In [ ]:
from pathlib import Path
import pandas as pd
from flypca.synthetic import generate_synthetic_trials
from flypca.lagpca import fit_lag_pca_for_trials, project_trial
from flypca.features import compute_feature_table
from flypca.cluster import cluster_features

trials, meta = generate_synthetic_trials(n_flies=3, trials_per_fly=8)
config = {
    "pre_s": 2.0,
    "post_s": 2.0,
    "lag_ms": 250,
    "n_components": 5,
    "threshold_k": 2.5,
    "smoothing": {"enable": True, "savgol_window_ms": 151, "savgol_poly": 3},
}
result = fit_lag_pca_for_trials(trials, config)
projections = {trial.trial_id: project_trial(trial, result) for trial in trials}
features = compute_feature_table(trials, config, result=result, projections=projections)
features = features.merge(meta, on=["trial_id", "fly_id"])
cluster_result = cluster_features(features.drop(columns=["reaction", "amplitude", "latency"]))
features.head()